In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import time
import sys
sys.path.append('../models/blend/')
from utils import customized_eval, HyperParameterTuning, KFoldValidation

In [ ]:
#---------------------
# setting
#---------------------
BINARY_SCENARIO = None
#---------------------
# load features
#---------------------
feature_dir = '../features/lazada_and_amazon/all_features.h5'
df = pd.read_hdf(feature_dir)
#---------------------
# label post-processing
#---------------------
if df.label.nunique() == 2: 
    BINARY_SCENARIO = True
    # binary class
    df['label'] = df.label.apply(lambda x: 1 if x == 2 else 0) # for customized f1 score inference of lgb
else:
    # multi-class(B, I or O)
    pass

In [ ]:
features = df.columns.tolist()[7:]
target = 'label'

In [ ]:
#-----------------------
# parameter tuning for session-level
#-----------------------
target = 'label'
n_splits = 5
pbounds = {
'num_leaves': (25, 50),
'lambda_l2': (0.0, 0.05),
'lambda_l1': (0.0, 0.05),
'min_child_samples': (20, 120),
'bagging_fraction': (0.5, 1.0),
'feature_fraction': (0.5, 1.0),
} # 6 parameters to tune

# Create objec HyperParameterTuning for helping us tuning
HP_tuning = HyperParameterTuning(train = df, features= features, target = target,
                                 n_splits = n_splits, params_bound = pbounds)

date_str = datetime.now().strftime('%Y-%m-%d_%H-%M')
s = time.time()
result = HP_tuning.param_tuning(init_points = 5, num_iter = 30)
e = time.time()
logging.info('It took {} mins'.format((e-s)/60.0))

result.points_to_csv('logs/param_tuning_for_lgb_{}.csv'.format(date_str), index = False)

logging.info(pp.pprint(result.res['max']['max_params']))
